In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import seaborn as sns

Настройка вывода данных

In [2]:
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)

Чтение дата сетов

In [3]:
df = pd.read_csv('train.csv')
df1 = pd.read_csv('valid.csv')

/var/folders/90/pj6yt0sd06b04mcq4xjmcl2h0000gn/T/ipykernel_3506/3447217829.py:1: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,131,132,133,135,136,139,140,141,142,143,144,145,146,147,148,149,151,152,155,156,157,158,159,160,161,163,164,165,167,168,169,171,172,173,174,175,176,177,178,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,223,224,227,239,240,241,242,243,244,245,246,459,460,461,467,468,469,515,516,523,524,531,532,539,540,547,548,555,556,563,564,565,567,571,572,573,575,579,580,581,582,583,587,588,595,596,603,604,611,612,613,615,795,796,797,799,1031,1032,1033,1034,1035,1036,1037,1039,1040,1041,1042,1043,1044,1045,1047,1048,1049,1050,1051,1052,1055,1056,1057,1058,1059,1060,1063,1064,1065,1067,1068,1069,1072,1073,1074,1075,1076,1077,1078,1172,1173,1174,1176,1177,1178,1179,1180,1181,1182,1275,1276,1277,1278,1279,1280,1281,1282,1447

Удаление дубликатов и столбцов с более чем 25% пропущенных значений


In [ ]:
df = df.drop_duplicates()

threshold = len(df) * 0.75
res = df.dropna(thresh=threshold, axis=1)
res.to_csv("res.csv", index=False)
print(res.info())

Просмотр количества пропущеных значений по столбцам

In [15]:
missing_value_count = res.isna().sum()
print(missing_value_count)


report_date      0
client_id        0
target           0
col1453          0
col1454          1
col2167         23
col2168         23
col2169         23
col2170         23
col2171         23
col2172         23
col2173         23
col2174         23
col2175         23
col2176         23
col2177         23
col2178         23
col2179         23
col2180         23
col2181         23
col2182         23
col2183         23
col2184         23
col2185         23
col2186         23
col2187         23
col2188         23
col2189         23
col2190         23
col2220         37
col2221         37
col2222         37
col2244        370
col2245        370
col2246        370
col2292         37
col2293         37
col2294         37
col2316         37
col2317         37
col2318         37
col2340         37
col2341         37
col2342         37
col2364         37
col2365         37
col2366         37
col2388         37
col2389         37
col2390         37
col2460        239
col2461        239
col2462     

Определяем категориальные столбцы которые должны закодировать
Потом кодируем данные
И заполняем оставшиеся пропущенные значения


In [ ]:
categorical_columns = res.select_dtypes(include=['object']).columns
res_encoded = pd.get_dummies(res, columns=categorical_columns)

res_encoded = res_encoded.fillna(0)

Создаем матрицу корреляции, выявляем и удаляем столбцы слишком сильно коррелирующие между собой

In [ ]:
corr_matrix = res_encoded.corr()
threshold = 0.9
high_corr_pairs = [(col1, col2) for col1 in corr_matrix.columns for col2 in corr_matrix.columns if col1 != col2 and abs(corr_matrix.loc[col1, col2]) > threshold]
to_drop = [pair[1] for pair in high_corr_pairs]
clean = res_encoded.drop(columns=to_drop)
clean.to_csv('clean.csv', index=False)

Выводим тепловую карту

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Тепловая карта корреляции')
plt.show()

Удаляем лишние булевые колонки появившиеся в ходе выше указаных преобразований

In [ ]:
for column in df.columns:
    if df[column].dtype == 'bool':
        df = df.drop(columns=column)

df.to_csv("clean1.csv", index=False)
print(df.info())

Удалим столбцы таблицы valid которых нету в таблице train

In [ ]:
valid1 = df1.reindex(columns=df.columns)
valid1.to_csv("valid1.csv", index=False)
valid1 = pd.read_csv('valid1.csv')
valid1.info()

Подготовка выборок для обучения и тестирования модели

In [ ]:
clean1 = pd.read_csv('clean1.csv')
valid1 = pd.read_csv('valid1.csv')

# разделение на признаки и целевую переменную train
X = clean1.drop('target', axis=1)
y = clean1['target']

# сохранение признаков и целевой переменной train
X.to_csv('features.csv', index=False)
y.to_csv('target.csv', index=False)


# разделение на признаки и целевую переменную valid
x_valid = valid1.drop('target', axis=1)
y_valid = valid1['target']

# сохранение признаков и целевой переменной valid
x_valid.to_csv('features_val.csv', index=False)
y_valid.to_csv('target_val.csv', index=False)

# загрузка данных train
X = pd.read_csv('features.csv')
y = pd.read_csv('target.csv')

# разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# загрузка данных valid
X_val = pd.read_csv('features_val.csv')
y_val = pd.read_csv('target_val.csv')

Создание модели, установка гиперпараметров

In [3]:
base_model = RandomForestRegressor(
    n_estimators=5,       # количество деревьев
    max_depth=5,           # глубина деревьев
    min_samples_split=25,  # минимальное количество образцов для разделения
    min_samples_leaf=15,    # минимальное количество образцов в листе
    random_state=42
)

model = BaggingRegressor(estimator=base_model, n_estimators=10, random_state=42)
model.fit(X_train, y_train)

Предсказание на тестовых данных иполучение метрик

In [6]:
# предсказание на тестовых данных
y_pred = model.predict(X_test)

# вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f'ROC-AUC Score: {roc_auc}')

# вывод графика ROC-AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

# вывод MSE
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

# вывод RMSE
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

# вывод графика MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

Предсказание на валидационных данных и получение метрик

In [18]:
# предсказание на данных valid
y_val_pred = model.predict(X_val)


# вычисление ROC-AUC
roc_auc = roc_auc_score(y_val, y_val_pred)
print(f'ROC-AUC Score: {roc_auc}')

# вывод графика ROC-AUC
fpr, tpr, thresholds1 = roc_curve(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

mse = mean_squared_error(y_val, y_val_pred)
print(f"MSE: {mse}")

rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

mae = mean_absolute_error(y_val, y_val_pred)
print(f"MAE: {mae}")

